In [1]:
%config IPCompleter.greedy=True
import pandas as pd
import re, string, unicodedata
import nltk

In [2]:
arrayHeaders = ["Area", "Dimension", "Respuesta"]
data = pd.read_csv("Data/example2.csv", header= None, names = arrayHeaders)

In [3]:
arrayAreas= []
for area in data["Area"]:
    area = area.split('\n')
    area = area[2]
    area = area.replace(':es: ','')
    area = area.replace('"','')
    area = area.replace('\r','')
    arrayAreas.append(area)
arrayAreas = pd.Series(arrayAreas)

In [4]:
arrayDimensiones = []
for dimension in data["Dimension"]:
    dimension = dimension.split('\n')
    dimension = dimension[2]
    dimension = dimension.replace(':es: ','')
    dimension = dimension.replace('"','')
    dimension = dimension.replace('\r','')
    arrayDimensiones.append(dimension)
    
arrayDimensiones = pd.Series(arrayDimensiones)

In [5]:
posAnswers = []
negAnswers = []

for resp in data["Respuesta"]:
    resp = resp.split('\n')
    resp_pos = resp[2].replace('       positive: ','')
    resp_pos = resp_pos.replace("'",'')
    resp_pos = resp_pos.replace('\r','')
    posAnswers.append(resp_pos)
    if len(resp)==4:
        resp_neg = resp[3].replace('       negative: ','')
        resp_neg = resp_neg.replace("'",'')
        resp_neg = resp_neg.replace('\r','')
        negAnswers.append(resp_neg)
    else:
        negAnswers.append('')
        
posAnswers = pd.Series(posAnswers)
negAnswers = pd.Series(negAnswers)

In [6]:
concatenaciónPositivas = pd.concat([arrayAreas,arrayDimensiones, posAnswers], axis = 1)
concatenaciónNegativas = pd.concat([arrayAreas,arrayDimensiones, negAnswers], axis = 1)


In [7]:
df = pd.concat([concatenaciónPositivas,concatenaciónNegativas],ignore_index=True).rename(columns = pd.Series(arrayHeaders))
def estadarizarDimensiones(token):
    token = re.sub("Anticorrupción", "Anti-corrupción", token)
    token = re.sub("Anti-Corrupción", "Anti-corrupción", token)
    
    token = re.sub("Comunidad ", "", token)
    token = re.sub("Concesionario ", "", token)
    token = re.sub("PEC ", "", token)
    token = re.sub("Proveedores ", "", token)
    token = re.sub("Compromiso ", "", token)
    token = re.sub("Operadores ", "", token)
    token = re.sub("Distribuidores ", "", token)
    token = re.sub("Operador ", "", token)
    
    token = re.sub("Cult. Ambiental", "Cultura Ambiental", token)
    token = re.sub("Cult. Amb.", "Cultura Ambiental", token)
    token = re.sub("Medio Ambiente", "Cultura Ambiental", token)

    token = re.sub("Entrenamiento & Desarrollo", "Entrenamiento y Desarrollo", token)

    token = re.sub("Engagement", "Compromiso", token)
    
    token = re.sub("Clientes", "Cliente", token)
    token = re.sub("Comunicación & Feedback", "Comunicación", token)
    
    token = re.sub("Salud & Seguridad", "saludyseguridad", token)
    token = re.sub("Salud", "saludyseguridad", token)
    token = re.sub("Seguridad", "saludyseguridad", token)
    token = re.sub("saludyseguridad", "Salud & Seguridad", token)

    token = token.replace('(','')
    token = token.replace(')','')
    return token
df["Dimension"] = df["Dimension"].apply(estadarizarDimensiones)
#set(df.Dimension)

In [10]:
len(df.groupby(['Dimension']).count().Respuesta )

58

In [11]:
df.groupby(['Area','Dimension']).count().Respuesta 

Area                  Dimension                   
Confianza General     Confianza General               27700
                      Información Adicional            2364
                      Infraestructura                    48
                      Innovación                       1446
Gobierno Corporativo  Anti-corrupción                 28790
                      Control Riesgos                    58
                      Cultura Ambiental                  68
                      Estrategia de Negocio             272
                      Estructura Directorio              66
                      Estructura Ejecutiva               22
                      Grupo Investor                     66
                      Integridad                      31096
                      Investors Day                      18
                      Legal                              26
                      Transparencia Información          54
                      Trato Accionistas          

In [9]:
df.to_csv("Data/formattedData.csv", sep=";", index=False)